In [ ]:
import os
import torch
import cv2
import easyocr
import csv
import logging
from ultralytics import YOLO


def detectar_direccion(track_id, label_name, x1, x2, frame):
    if (
        track_id not in datos[label_name]["from_front"]
        and track_id not in datos[label_name]["to_front"]
    ):
        if label_name == "person" or label_name == "bicycle":
            if (x2) < frame.shape[1] * 0.2 or (frame.shape[1]*0.7< (x2) < frame.shape[1] * 0.95):
                datos[label_name]["to_front"].append(track_id)
                return "viene de atras"
            else:
                datos[label_name]["from_front"].append(track_id)
                return "viene de frente"
        else:
            if (x2) < frame.shape[1] * 0.75:
                datos[label_name]["from_front"].append(track_id)
                return "viene de frente"
            elif (x2) > frame.shape[1] * 0.9:
                datos[label_name]["to_front"].append(track_id)
                return "viene de atras"
    else:
        if track_id in datos[label_name]["from_front"]:
            return "viene de frente"
        else:
            return "viene de atras"


# Configura el logger para suprimir mensajes
logging.getLogger("ultralytics").setLevel(logging.WARNING)

# Verifica si se puede usar una GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU disponible: ", torch.cuda.is_available())

# Configuración de OCR
reader = easyocr.Reader(["es"])

classNames = ["person", "bicycle", "car", "motorbike", "bus"]

# Cargar los modelos YOLO
model_general = YOLO("yolo11n.pt")  # Modelo para detectar personas y vehículos

# Configuración del archivo de video y CSV
filename = "C0142.MP4"
output_video_filename = "resultado_deteccion.mp4"
csv_filename = "resultados.csv"

# Inicializar el video y el escritor
cap = cv2.VideoCapture(filename)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(
    cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
)
out_video = cv2.VideoWriter(
    output_video_filename, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height)
)


# Variables de conteo
datos = {name: {"from_front": [], "to_front": []} for name in classNames}

# Procesar el video con detección general
results = model_general.track(source=filename, show=True, stream=True, verbose=False)

frame_count = 0

# Almacenar matrículas y sus IDs para el seguimiento con su confianza
matrículas_seguimiento = {}

# Iterar sobre cada resultado por frame
for frame_result in results:
    frame_count += 1
    frame = frame_result.orig_img

    # Imprimir fotograma actual y total de fotogramas
    print(f"Procesando fotograma {frame_count}")

    # Obtener detecciones y agregar track_id de cada objeto
    for box in frame_result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        score = box.conf[0].item()
        label = int(box.cls[0])
        track_id = int(box.id[0]) if box.id is not None else -1

        if score >= 0.4:  # Umbral de confianza
            # Verificar si el label es válido para YOLO
            if 0 <= label < len(classNames):
                label_name = classNames[label]

                direccion = detectar_direccion(track_id, label_name, x1, x2, frame)

                # Dibujar el rectángulo y mostrar el ID de seguimiento en el objeto
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(
                    frame,
                    f"ID: {track_id} Direct: {direccion}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 255, 0),
                    2,
                )
                cv2.putText(
                    frame,
                    label_name,
                    (x1, y1 - 25),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 255, 255),
                    2,
                )

    # Guardar el frame procesado en el archivo de video

    out_video.write(frame)

# Mostrar conteo de clases
print("Conteo total de objetos:")
for object, data in datos.items():
    print(
        f"{object}: \n\t{len(data['from_front'])} vienen de frente, \n\t{len(data['to_front'])} vienen de atras, \n\t total: {len(data['from_front']) + len(data['to_front'])}"
    )

# Liberar recursos
cap.release()
out_video.release()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


GPU disponible:  False
WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'

Procesando fotograma 1
Procesando fotograma 2
Procesando fotograma 3
Procesando fotograma 4
Procesando fotograma 5
Procesando fotograma 6
Procesando fotograma 7
Procesando fotograma 8
Procesando fotograma 9
Procesando fotograma 10
Procesando fotograma 11
Procesando fotograma 12
Procesando fotograma 13
Procesando fotograma 14
Procesando fotograma 15
Procesando fotograma 16
Procesando fotograma 17
Procesando fotograma 18
Procesando fotograma 19
Procesando fotograma 20
Procesando fotograma 21
Procesando fotograma 22
Procesando fotograma 23
Procesando f